# Ejemplos de Optimización


### **Ejercicio de maximización rentabilidad por dividendos:**

_Objetivo:_ Maximizar la rentabilidad por dividendos de una cartera teniendo en cuenta que la exposición a moneda extranjera debe ser menos o igual al 40%:

**Activos:**

| Activo | Dividendo | % Ingresos Extranjero |
|--------|-----------|-----------------------|
| SAN    | 3.67%     | 60%                   |
| REE    | 4.2%      | 10%                   |
| BBVA   | 5.6%      | 50%                   |
| REPSOL | 5%     | 25%                   |


**Entrega:**
Determinar las proporciones óptimas de inversión para cada activo, calculando la rentabilidad total y la exposición a moneda extranjera.

In [2]:
import numpy as np
import pandas as pd
import cvxpy as cp

In [2]:
# datos almacenados en arrays
dividendos = np.array([3.67, 4.2, 5.6, 5])
ingresos = np.array([60, 10, 50, 25])

In [3]:
# creacion de variables de decisión
pesos = cp.Variable(dividendos.shape[0])

In [5]:
# construcción de función objetivo
objective = cp.sum(cp.multiply(pesos, dividendos))

In [50]:
# lista de expresiones representando las restricciones
constraints = [
    cp.sum(pesos) == 1.0,
    cp.sum(cp.multiply(ingresos, pesos)) <= 40,
    pesos >= 0.0
]

In [51]:
# instancia de un problema en CVXPY
problem = cp.Problem(cp.Maximize(objective), constraints)

In [59]:
# resolución del problema y valor optimo
result = problem.solve()
print(f'La rentabilidad por dividendo obtenida es: {result:.2f}%')

La rentabilidad por dividendo obtenida es: 5.36%


In [54]:
# valor de las variables en el máximo de la función objetivo
pesos.value

array([1.25638459e-10, 6.71469679e-10, 5.99999999e-01, 4.00000000e-01])

In [56]:
pd.Series(100*pesos.value.round(2), index=['SAN', 'REE','BBVA','REPSOL'])

SAN        0.0
REE        0.0
BBVA      60.0
REPSOL    40.0
dtype: float64

### Apartado adicional
- Resolver el problema anterior si además no se puede invertir más del 30% en un solo activo.


In [60]:
# añadimos la nueva restricción
constraints.append(pesos <= 0.3)

In [62]:
# resolución del problema y valor optimo
problem = cp.Problem(cp.Maximize(objective), constraints)
result = problem.solve()
print(f'La rentabilidad por dividendo obtenida es: {result:.2f}%')

La rentabilidad por dividendo obtenida es: 4.81%


In [63]:
pd.Series(100*pesos.value.round(2), index=['SAN', 'REE','BBVA','REPSOL'])

SAN       10.0
REE       30.0
BBVA      30.0
REPSOL    30.0
dtype: float64

---

### Ejercicio 2: Selección Óptima de Proyectos con Presupuesto y Riesgo Limitados

**Contexto:** Eres un gestor de inversiones con un presupuesto limitado y necesitas decidir en qué proyectos invertir. Cada proyecto tiene un costo asociado, un retorno esperado y un nivel de riesgo.

**Objetivo:** Maximizar el retorno total esperado de tu inversión, respetando las restricciones de presupuesto y riesgo.

**Datos:**

| Proyecto | Costo | Retorno Esperado | Riesgo |
|----------|-------|------------------|--------|
| 1        | 100   | 10               | 0.2    |
| 2        | 200   | 25               | 0.6    |
| 3        | 150   | 20               | 0.4    |
| 4        | 120   | 15               | 0.3    |
| 5        | 180   | 30               | 0.7    |

- Presupuesto total: 400.
- Riesgo máximo total permitido: 1.2.

**Restricciones Adicionales:**
- Inversión parcial en proyectos permitida (puedes invertir cualquier fracción de los costos).
- Dependencia entre proyectos: la inversión en el proyecto 1 excluye la inversión en el proyecto 2.

**Tarea:**
Utiliza `cvxpy` para formular y resolver el problema de optimización. Encuentra la fracción óptima de inversión en cada proyecto.


In [3]:
# Datos del problema
costos = np.array([100, 200, 150, 120, 180])  # Costo de cada proyecto
retornos = np.array([10, 25, 20, 15, 30])     # Retorno esperado de cada proyecto
riesgos = np.array([0.2, 0.6, 0.4, 0.3, 0.7]) # Riesgo de cada proyecto
presupuesto = 400
riesgo_maximo = 1.2

In [4]:
# Variables de decisión (fracción de inversión en cada proyecto)
x = cp.Variable(len(costos))

In [8]:
# Restricciones
constraints = [
    cp.sum(x @ costos) <= presupuesto,  # No superar el presupuesto
    cp.sum(cp.multiply(x, riesgos)) <= riesgo_maximo, # No superar el riesgo máximo
    x >= 0,  # No se permiten valores negativos
    x <= 1,  # No se puede invertir más del 100% en un proyecto
    x[0] + x[1] <= 1  # Dependencia entre proyectos 1 y 2
]

**Explicación de la Restricción $x[0] + x[1] <= 1$**

La restricción $x[0] + x[1] \leq 1$ establece una relación especial entre los proyectos 1 y 2 en nuestro problema de optimización:

- **Variables $x[0]$ y $x[1]$:** Representan la fracción de inversión en los proyectos 1 y 2, respectivamente. Estas fracciones pueden variar de 0 (sin inversión) a 1 (inversión completa).

- **Restricción $x[0] + x[1] \leq 1$:** Implica que la suma total de las inversiones en ambos proyectos no puede superar el 100%. Esto se puede interpretar de dos maneras:

  1. **Exclusión Mutua:** Si se invierte completamente en uno de los proyectos (por ejemplo, $x[0] = 1$), entonces no se puede invertir nada en el otro ($x[1] = 0$), y viceversa.

  2. **Inversión Parcial Compartida:** Se permite la inversión parcial en ambos proyectos siempre y cuando la suma total de estas inversiones no exceda el 100%. Ejemplo: invertir el 50% en cada proyecto ($x[0] = 0.5$ y $x[1] = 0.5$).

Aplicaciones Prácticas

- **Recursos Compartidos o Limitados:** Esta restricción es útil cuando los dos proyectos comparten recursos limitados (capital, personal, tiempo) y no es posible comprometerse completamente con ambos.

- **Decisiones Estratégicas:** Puede reflejar una decisión estratégica donde invertir en un proyecto excluye o limita la inversión en otro debido a factores como la competencia de mercado o el enfoque estratégico de la empresa.


In [13]:
# Función objetivo
objetivo = cp.Maximize(x @ retornos)

# Definir y resolver el problema
problema = cp.Problem(objetivo, constraints)
problema.solve(solver='CLARABEL')

# Resultados
print("Inversión óptima en cada proyecto:", x.value)


Inversión óptima en cada proyecto: [2.94117649e-02 4.32865867e-11 1.00000000e+00 1.00000000e+00
 7.05882353e-01]


In [18]:
inversiones = [round(x,3) if x > 10**-3 else 0 for x in x.value]
print("Inversión óptima en cada proyecto:", inversiones)

Inversión óptima en cada proyecto: [0.029, 0, 1.0, 1.0, 0.706]


In [12]:
cp.installed_solvers()

['CLARABEL', 'ECOS', 'ECOS_BB', 'OSQP', 'SCIPY', 'SCS']

---

### Apartados adicionales
- Resolver el problema anterior si además quieres una rentabilidad mínima del 14%

In [133]:
constraints.append(retornos_esperados @ pesos >= 0.14)

In [134]:
# Problema de optimización
problema = cp.Problem(objetivo, constraints)

# Resolviendo el problema
resultado = problema.solve()

# Resultados
pesos_optimos = pesos.value

In [91]:
(retornos_esperados @ pesos_optimos).round(4)

0.11

In [135]:
print(f'El riesgo de la cartera es: {riesgo.value:.2f}')
print(f'La rentabilidad de la cartera es: {100*(retornos_esperados @ pesos_optimos).round(4)}%')
print(f'Los pesos que hacen que la cartera tenga el menor riesgo son: {100*pesos_optimos.round(2)}')

El riesgo de la cartera es: 1.25
La rentabilidad de la cartera es: 14.000000000000002%
Los pesos que hacen que la cartera tenga el menor riesgo son: [62. -0. -0. 38.]


- ¿Y si la rentabilidad mínime es del 18%?

In [136]:
# Restricciones
constraints = [cp.sum(pesos) == 1, 
               pesos >= 0,
               retornos_esperados @ pesos >= 0.18]


# Problema de optimización
problema = cp.Problem(objetivo, constraints)

# Resolviendo el problema
resultado = problema.solve()

# Resultados
pesos_optimos = pesos.value

In [137]:
print(f'Los pesos que hacen que la cartera tenga el menor riesgo son: {100*pesos_optimos.round(2)}')

AttributeError: 'NoneType' object has no attribute 'round'

In [101]:
resultado

inf

Las restricciones establecidas en este problema de optimización hacen que no tenga solución.

Dado que el rendimiento más alto de los activos es del 17%, es imposible lograr una rentabilidad del 18% con una cartera formada por estos activos.

---

### **Ejercicio de Optimización de Portafolio con Máxima Rentabilidad dado un Riesgo**

**Objetivo**: Maximizar la rentabilidad de un portafolio de inversión asegurando un riesgo máximo.

### Datos:
- **Retornos Esperados**: Un array `retornos_esperados` que contiene el retorno esperado para cada activo.
- **Matriz de Covarianza**: `matriz_cov`, describiendo la covarianza entre los activos y el riesgo combinado.


### Restricciones:
1. No se permiten posiciones cortas

In [138]:
# Datos de retornos y covarianzas

retornos_esperados = np.array([0.17, 0.10, 0.07, 0.09])  # Ejemplo de retornos esperados
matriz_cov = np.array([[1.83977374, 1.23002575, 1.59282297, 0.69409837],
       [1.23002575, 1.45345954, 1.7548078 , 1.31477996],
       [1.59282297, 1.7548078 , 2.14425197, 1.55568552],
       [0.69409837, 1.31477996, 1.55568552, 1.46502412]])

In [139]:
# Variable de decisión
pesos = cp.Variable(4)

In [140]:
# Función objetivo
rentabilidad = retornos_esperados @ pesos
objetivo = cp.Maximize(rentabilidad)

In [149]:
# Restricciones
riesgo = cp.quad_form(pesos, matriz_cov)

constraints = [cp.sum(pesos) == 1, 
               pesos >= 0,
               riesgo <= 1.20]

riesgo = cp.quad_form(pesos, matriz_cov)

In [150]:
# Problema de optimización
problema = cp.Problem(objetivo, constraints)

# Resolviendo el problema
resultado = problema.solve()

# Resultados
pesos_optimos = pesos.value

In [151]:
resultado

0.13444668904708934

In [152]:
print(f'El riesgo de la cartera es: {riesgo.value:.2f}')
print(f'La rentabilidad de la cartera es: {100*(retornos_esperados @ pesos_optimos).round(4)}%')
print(f'Los pesos que hacen que la cartera tenga el menor riesgo son: {100*pesos_optimos.round(2)}')

El riesgo de la cartera es: 1.20
La rentabilidad de la cartera es: 13.44%
Los pesos que hacen que la cartera tenga el menor riesgo son: [56.  0.  0. 44.]


---

### **Ejercicio de Optimización de Portafolio usando ratio de sharpe**

## Problema de Optimización del Ratio de Sharpe

**Objetivo**: Maximizar el ratio de Sharpe cuadrático de un portafolio de inversión.

### Datos:
- **Retornos Esperados**: Vector `retornos_esperados` con el retorno esperado de cada activo.
- **Matriz de Covarianza**: Matriz `matriz_cov` que representa la covarianza entre los rendimientos de los activos.
- **Tasa Libre de Riesgo**: Valor `tasa_libre_riesgo`, representando la tasa de retorno sin riesgo.

### Solución:
Buscamos los pesos que conforman la cartera con el mayor ratio de Sharpe, optimizando así la relación entre el retorno esperado y la volatilidad del portafolio.


In [109]:
# Datos de ejemplo
retornos_esperados = np.array([0.12, 0.10, 0.07, 0.09])

matriz_cov = np.array([[1.6768397 , 0.66434839, 1.26288857, 1.85128714],
                        [0.66434839, 0.69280958, 0.87199711, 0.55437544],
                        [1.26288857, 0.87199711, 1.92091975, 1.5573455 ],
                        [1.85128714, 0.55437544, 1.5573455 , 2.51120125]])

tasa_libre_riesgo = 0.03

In [110]:
# Variable de decisión
pesos = cp.Variable(4)

In [111]:
# Función objetivo (Ratio de Sharpe)
retorno_portafolio = retornos_esperados @ pesos
volatilidad_portafolio = cp.sqrt(cp.quad_form(pesos, matriz_cov))
ratio_sharpe = (retorno_portafolio - tasa_libre_riesgo) / volatilidad_portafolio

In [112]:
# Restricciones
constraints = [cp.sum(pesos) == 1, pesos >= 0]

In [113]:
# Definición del problema
problema = cp.Problem(cp.Maximize(ratio_sharpe), constraints)

# Intentando resolver el problema
try:
    problema.solve()
except Exception as e:
    print("Error al resolver el problema:", e)

Error al resolver el problema: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
power(QuadForm(var710, [[1.68 0.66 1.26 1.85]
 [0.66 0.69 0.87 0.55]
 [1.26 0.87 1.92 1.56]
 [1.85 0.55 1.56 2.51]]), 0.5)
